# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [179]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size. 


In [180]:
def merge(left, right, symbols_dict):
    left_keys = list(left[0])
    right_keys = list(right[0])

    for key in left_keys:
        symbols_dict[key].append('1')
    for key in right_keys:
        symbols_dict[key].append('0')

    merged_keys = left[0]+right[0]
    merged_values = left[1]+right[1]

    return (merged_keys, merged_values)
    

def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities  
    
    Arguments:
    ----------
    probability_dict:
      - keys: symbol as character or string
      - values: probability of the symbol as a float or double
      Example: {"A": 0.25, "B":0.5,"C":0.125,"D":0.125}
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """

    # Convert the dictionary to a list of tuples
    probability_list = list(probability_dict.items())

    symbols_dict = {}
    for key in probability_dict.keys():
        symbols_dict[key] = []

    for i in range(len(probability_list)):
        probability_list[i] = ([probability_list[i][0]], probability_list[i][1])

    trees = []
    for i in range(len(probability_list)):
        trees.append(probability_list[i])

    # merge the trees until there is only one tree left by using the merge function
    while len(trees) > 1:
        trees.sort(key=lambda x: x[1])
        left = trees.pop(0)
        right = trees.pop(0)
        merged = merge(left, right, symbols_dict)
        trees.append(merged)

    # reverse the order of the codewords and convert the list to a string
    for key in symbols_dict.keys():
        symbols_dict[key] = symbols_dict[key][::-1]
        symbols_dict[key] = ''.join(symbols_dict[key])

    return symbols_dict

In [181]:
prob_dict = {"A":0.05,"B":0.10,"C":0.15,"D":0.15,"E":0.2,"F":0.35}
print(Huffman_code(prob_dict))

{'A': '111', 'B': '110', 'C': '011', 'D': '010', 'E': '10', 'F': '00'}


### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course. 

In [182]:
def LZ_online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols  
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    
    Return:
    -------
    - dictionary : the computed dictionnary in the form:
        - keys: symbol as character or string
        - values: associated codeword as a tuple composed of the entry index (integer) and a binarized adress with one appended symbol (character or string)
        Example: {'': (0, ''), '0': (1, '0'), '1': (2, '01'), '00': (3, '010'), '10': (4, '100')}
    - encoded_sequence : the encoded sequence in the string format
    """
    dictionary = {'': (0, '')}
    encoded_sequence = ''
    next_index = 1
    i = 0

    while i < len(sequence):

        prefix = ''
        buf_prefix = ''
        j = i
        next_index_log = np.ceil(np.log2(next_index))

        while prefix in dictionary and j < len(sequence):
            buf_prefix = prefix
            prefix += sequence[j]
            j += 1

        if prefix not in dictionary and len(prefix) == 1:

            prefix = buf_prefix
            prefix_dic = dictionary[prefix]

            prefix_index = '{0:b}'.format(prefix_dic[0])
            encoded_sequence += sequence[j - 1].zfill(int(next_index_log + 1)) #format(entry_index, 'b').zfill(num_bits) + symbol #prefix_dic[1] + sequence[j - 1]

            dictionary[prefix + sequence[j - 1]] = (next_index, sequence[j - 1].zfill((int(next_index_log + 1))))
            next_index += 1
            i = j
        if prefix not in dictionary and len(prefix) > 1:
            prefix = buf_prefix
            prefix_dic = dictionary[prefix]

            prefix_index = '{0:b}'.format(prefix_dic[0])
            encoded_sequence += prefix_index.zfill(int(next_index_log)) + sequence[j - 1] #format(entry_index, 'b').zfill(num_bits) + symbol #prefix_dic[1] + sequence[j - 1]

            dictionary[prefix + sequence[j - 1]] = (next_index, prefix_index.zfill(int(next_index_log)) + sequence[j - 1])
            next_index += 1
            i = j
        else:
                i = j

    return dictionary, encoded_sequence

In [183]:
t_seq = '1011010100010'
u_seq = '100011101100001000010'

dict, u_lz = LZ_online(t_seq)
print(dict)
print(u_lz)
print(u_seq == u_lz)

print(len(t_seq)*8)
print(len(u_lz))

{'': (0, ''), '1': (1, '1'), '0': (2, '00'), '11': (3, '011'), '01': (4, '101'), '010': (5, '1000'), '00': (6, '0100'), '10': (7, '0010')}
100011101100001000010
True
104
21


### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [184]:
def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size
    
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer
    
    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """

In [185]:
# [Locked Cell] Evaluation of your functions by the examiner. 
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression


In [186]:
# Write here your codes for questions 5 to 15 (you may delete this comment)

## Channel coding

In [187]:
# Write here your codes for questions 16 to 21 (you may delete this comment)